In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ktrain
!pip install git+https://github.com/amaiya/eli5@tfkeras_0_10_1

import ktrain
from ktrain import text
import numpy as np
import glob
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.3 MB 1.3 MB/s 
     |████████████████████████████████| 981 kB 62.4 MB/s 
     |████████████████████████████████| 263 kB 65.0 MB/s 
     |████████████████████████████████| 3.8 MB 53.0 MB/s 
     |████████████████████████████████| 1.3 MB 53.2 MB/s 
     |████████████████████████████████| 468 kB 64.4 MB/s 
     |████████████████████████████████| 880 kB 69.2 MB/s 
     |████████████████████████████████| 6.6 MB 48.6 MB/s 
     |████████████████████████████████| 120 kB 65.1 MB/s 
  Created wheel for ktrain: filename=ktrain-0.31.7-py3-none-any.whl size=25312842 sha256=b20fcec3df5e069c49c9f6be0d4937e6bf4f54b9216c017676beeca10523c17b
  Stored in directory: /root/.cache/pip/wheels/e5/98/8e/ce355dcb92451e85fab93f7ea2da068843e93e703928cd06fb
  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33517 sha256=1c0acad09d5ff17a496fda10

In [ ]:
trainDf = pd.read_csv('/content/drive/MyDrive/AI project/Test.csv')
valDf = pd.read_csv('/content/drive/MyDrive/AI project/Val.csv')

In [ ]:
trainDf

,Data,Label
0,"স্বাস্থ্যবান হতে চাই , আমি বয়সের তুলনায় অনেক ব...",0
1,ভাইয়া নতুন ভিডিও আসে না কেন,0
2,সৌরভ গাঙ্গুলী ছাড়া দাদাগিরি কখনো জমে উঠত না,0
3,ক্রিকেট কে বাচাতে হলে পাপকে অতিশিগ্রিই তাকেও গ...,2
4,আমিতো সেই ঝালপ্রিয়ো মানুষ,1
...,...,...
1581,ঐ পুলিশ কুওারবাচচাদের গুলিকরে মেরে ফেলা উচিত,2
1582,খিচুড়ি পর্ব মারাত্মক । বাকি টা দেখি তারপর কমেন্ট,1
1583,পাশের ফ্রেন্ডটা কি ছেলে না মেয়ে,2
1584,মদ লকমান না খেলেও আমরা খাই এমনটাই মনে হচ্ছে,2


In [ ]:
valDf

,Data,Label
0,আর আমার খুবেই আনন্দ লাকছে ভাইটি চাকরি পেয়েছে,1
1,ভাই আমাদের আগের মেয়র আনিচুল হক নাই যে আমাদের ক...,2
2,আমি মার্ক ওয়েন আর সনির বিশাল ভক্ত । একটা সময় ভ...,1
3,৩ মাস না যেতেই একেকজন ফুলে ফেপে আলুর দম,2
4,"বাংলাদেশের পুলিশ হলো নিরীহ মানুষের যম , আর অত্...",2
...,...,...
1562,আর বিচার হবে বলে মনে হয় না,2
1563,"ভাই রাতে গেলে পাবেন , ০১৮৬২২৮৪৪৯৫ অথবা ০১৮৬২৮০...",1
1564,আমাকে যে দিন ( যেই তারিখ এ ) ডক্টর এ্যাপয়েনমে...,1
1565,আরে ভাই এখান থেকে তো রাজনীতি করে বড় হচ্ছে আর য...,2


In [ ]:
train_body, test_body, train_cluster_id, test_cluster_id = trainDf['Data'].values, valDf['Data'].values, trainDf['Label'].values, valDf['Label'].values

In [ ]:
MODEL_NAME = 'csebuetnlp/banglabert'
t = text.Transformer(MODEL_NAME, maxlen=200)

Downloading:   0%|          | 0.00/586 [00:00<?, ?B/s]

In [ ]:
X_train = list(train_body)
y_train = list(train_cluster_id)
X_test = list(test_body)
y_test = list(test_cluster_id)

y_train = [str(x) for x in train_cluster_id]
y_test = [str(x) for x in test_cluster_id]

trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

preprocessing train...
language: bn
train sequence lengths:
	mean : 15
	95percentile : 36
	99percentile : 44


Downloading:   0%|          | 0.00/119 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/516k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 16
	95percentile : 37
	99percentile : 47


In [ ]:
learning_rate = 1e-4
epochs = 3

model = t.get_classifier()

learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=16)

learner.fit_onecycle(learning_rate, epochs)

/usr/local/lib/python3.7/dist-packages/ktrain/text/preprocessor.py:1171: UserWarning: Could not load a Tensorflow version of model. (If this worked before, it might be an out-of-memory issue.) Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. You will need PyTorch installed for this.
  "Could not load a Tensorflow version of model. (If this worked before, it might be an out-of-memory issue.) "




begin training using onecycle policy with max lr of 0.0001...
Epoch 1/3
100/100 [==============================] - 89s 744ms/step - loss: 1.0065 - accuracy: 0.4994 - val_loss: 0.8200 - val_accuracy: 0.6471
Epoch 2/3
100/100 [==============================] - 72s 715ms/step - loss: 0.8786 - accuracy: 0.6211 - val_loss: 0.8949 - val_accuracy: 0.6356
Epoch 3/3
100/100 [==============================] - 72s 715ms/step - loss: 0.6930 - accuracy: 0.7188 - val_loss: 0.8171 - val_accuracy: 0.6631


In [ ]:
learner.validate(class_names=t.get_classes())

              precision    recall  f1-score   support

           0       0.44      0.53      0.48       354
           1       0.73      0.74      0.74       623
           2       0.76      0.66      0.71       590

    accuracy                           0.66      1567
   macro avg       0.65      0.64      0.64      1567
weighted avg       0.68      0.66      0.67      1567



array([[188, 107,  59],
       [ 98, 461,  64],
       [138,  62, 390]])

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t)
predictor.save('combined_model')